In [5]:
import pandas as pd
from os.path import join
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression, make_low_rank_matrix
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from gprEmbedding import SubspaceKernel, EmbeddingRBF
from sklearn.gaussian_process import GaussianProcessRegressor

import pandas as pd
import numpy as np

In [6]:
path = 'code_example_data'
trainX = pd.read_csv(join(path, 'trainX.csv'), index_col=0)
trainY = pd.read_csv(join(path, 'trainY.csv'), index_col=0)
    
testX = pd.read_csv(join(path, 'testX.csv'), index_col=0)
testY = pd.read_csv(join(path, 'testY.csv'), index_col=0)

In [7]:
trainX.head()

,VCD,Glc,Gln,Amm,Lac,product,reactor_temperature,pH,reactor_volume,stirring_rate,inital_volume,total_run_time,product_is_clever_lemon,product_is_relaxed_soup,product_is_novel_brick,product_is_savage_yogurt,product_is_forgiving_crumble,product_is_NNcurious_pretzel
0,0.497567,47.187399,8.418417,21.040312,19.233585,554.786905,36.603477,6.848805,0.240,168.361456,0.25,336.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2.149359,15.519523,9.419511,9.228290,69.201177,593.394556,36.838207,6.973305,0.250,170.309117,0.25,336.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.645692,15.497978,0.637340,5.118473,70.339972,747.287453,36.962887,6.705709,0.235,164.639278,0.25,336.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.355481,42.822897,5.192189,0.100000,0.100000,0.000000,37.034637,6.950299,0.250,191.907496,0.25,336.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.215053,53.815468,8.999396,4.476596,13.620892,22.769994,36.878436,6.981446,0.250,182.791211,0.25,336.0,0.0,0.0,0.0,1.0,0.0,0.0


# Example Data
The example data set is concerned with cell processes in biology. For different cell lines (or products) we predict the growth rate of the cells. 

The target `y` is the current rate of cell growth.  
The features `X` contain real valued information like Temperature, pH or current concentrations of nutrients in the brooth. Additionally,
the identity of the product is represented with a one-hot vector in the last 6 columns. 

# The Method
We implemented a custom kernel `EmbeddingRBF` that will essentially replace the one-hot vectors in the raw data with a learned embedding vector. It implements the kernel function

$$k(x, y) =  exp(- ||Wx- Wy||^2 ) $$

where W is a learned matrix. `x` and `y` are one-hot vectors. This kernel is applied to the one-hot vector that encodes the product identity and multiplied with an RBF kernel which is applied to the remaining features. This is equivalent to replacing the one-hot vectors with the learned embeddings (correct columns in W) and then feeding them together with the normal features into a normal RBF kernel.

In [ ]:
n_features = 12 # the number of normal features
n_products = 6 # the number of products 
embedding_dimension = 2 # can be choosen freely

prod_col_names = trainX.columns.values[-n_products:].tolist() # the names of the columns encoding the product identity
prod_col_names 

In [9]:
def rmse(hat, target):
    target = target.values
    sq = (hat-target)**2
    return np.sqrt(np.mean(sq))/np.std(target)

def MDe(hat, target):
    target = target.values
    sq = (hat-target)**2
    return np.sqrt(np.median(sq))/np.std(target)

In [10]:
# Apply RBF kernel to the normal features
raw_normal_feature_kernel = RBF(length_scale = [1.0]*n_features)
# The SubspaceKernel ensures that the RBF kernel is only applied to the first 10 features
normal_feature_kernel = SubspaceKernel(raw_normal_feature_kernel, ids_to_apply=np.arange(0, n_features))

# Apply Embedding kernel to the last 6 features, that contain a one-hot representation of the product
raw_embedding_kernel = EmbeddingRBF.make4EntityEmbed(n_entities=n_products, embedding_dimension=embedding_dimension)
# Use SubspaceKernel to indicate that the one-hot feature representation is contained in the last 6 columns
embedding_kernel = SubspaceKernel(raw_embedding_kernel, ids_to_apply=np.arange(n_features, n_features+n_products))

# Combine the kernel and allow for Noise
full_kernel = 1**2*normal_feature_kernel*embedding_kernel + WhiteKernel()

In [ ]:
embedding_gp = GaussianProcessRegressor(kernel=full_kernel, n_restarts_optimizer=1, normalize_y=True)

embedding_gp.fit(trainX, trainY)

In [ ]:
y_hat_embed = embedding_gp.predict(testX)

In [ ]:
print('standardised root mean square error: {:.4f}'.format(rmse(hat=y_hat_embed, target=testY)))
print('standardised median error: {:.4f}'.format(MDe(hat=y_hat_embed, target=testY)))

# Recovering the Embeddings